# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [335]:
from xml.etree import ElementTree as ET
import pandas as pd

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [336]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [337]:
# print names of all countries
for child in document_tree.getroot():
    print(child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [338]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print('* ' + element.find('name').text + ':'),
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print(capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [339]:
document = ET.parse( './data/mondial_database.xml' )

### 1. 10 countries with the lowest infant mortality rates

In [340]:
# create dataframe to store country names and infant mortality rates
df = pd.DataFrame(columns=('country', 'infmort'))

In [341]:
# iterate through countries and add names and infant mortality rates to dataframe
i = 0
root = document.getroot()
for child in root.findall('country'):
    if child.find('infant_mortality') != None:
        infmort = float(child.find('infant_mortality').text)
        name = child.find('name').text
        df.loc[i] = name, infmort
        i += 1

In [342]:
# find 10 countries with lowest mortality rates
df.sort_values('infmort').head(10)

,country,infmort
36,Monaco,1.81
90,Japan,2.13
109,Bermuda,2.48
34,Norway,2.48
98,Singapore,2.53
35,Sweden,2.60
8,Czech Republic,2.63
72,Hong Kong,2.73
73,Macao,3.13
39,Iceland,3.15


### 2. 10 cities with the largest population

In [357]:
# create dataframe to store country names and populations
df = pd.DataFrame(columns=('country', 'pop'))

In [358]:
# iterate through countries and add names and populations to dataframe
i = 0
root = document.getroot()
for child in root.findall('country'):
    name = child.find('name').text
    pop = int(child.find('population').text)
    df.loc[i] = name, pop
    i += 1

In [359]:
# find 10 countries with largest population
df.sort_values('pop', ascending=False).head(10)

,country,pop
55,China,543776080.0
67,India,238396327.0
120,United States,157813040.0
23,Russia,102798657.0
98,Japan,82199470.0
88,Indonesia,72592192.0
11,Germany,68230796.0
176,Brazil,53974725.0
53,United Kingdom,50616012.0
7,France,40502513.0


### 3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

In [367]:
# create dataframe to store country names, ethnic groups, and populations
df = pd.DataFrame(columns=('ethgroup', 'pop'))

In [368]:
# iterate through countries and add ethnic groups and populations to dataframe
i = 0
root = document.getroot()
for child in root.findall('country'):
    if child.find('ethnicgroup') != None:
        ethgroup = child.find('ethnicgroup').text
        pop = float(child.find('population').text)
    
    df.loc[i] = ethgroup, pop
    i += 1

In [369]:
# add up populations of each ethnic group
df = df.groupby('ethgroup')[['pop']].sum() 

In [370]:
# identify 10 groups with largest overall populations
df.sort_values('pop', ascending=False).head(10)

,pop
ethgroup,
Han Chinese,543776080.0
Japanese,328797880.0
European,297887587.0
Dravidian,238396327.0
Javanese,217776576.0
Russian,102798657.0
German,96828641.0
Arab,64293999.0
African,51894052.0


### 4. Name and country of a) longest river, b) largest lake and c) airport at highest elevation

#### Country with longest river

Construct dictionary of country IDs and names

In [371]:
di = pd.DataFrame(columns=('countryID', 'country'))

i = 0
root = document.getroot()
for child in root.findall('country'):
    countryID = child.get('car_code')
    country = child.find('name').text
    di.loc[i] = countryID, country
    i += 1

# convert di to dictionary
di = di.groupby(['countryID']).first()
di = di['country'].sort_index()

Construct dataframe of country IDs, country names, river names, and river lengths

In [372]:
df = pd.DataFrame(columns=('countryID', 'country', 'river', 'length'))

In [373]:
# add country code, river name, and river length to dataframe:
i = 0
root = document.getroot()
for child in root.findall('river'):
    name = child.find('name').text
    countryID = child.get('country')
    # if length is missing, enter into dataframe as 0
    if child.find('length') == None:
        length = 0
    else:
        length = float(child.find('length').text)
    df.loc[i] = countryID, 'NA', name, length
    i += 1

Identify country with longest river

In [374]:
df = df.sort_values('length', ascending=False).head(1)

# fill in country name
df.set_index('countryID', inplace = True)

df['country'].update(pd.Series(di))
df

,country,river,length
countryID,,,
CO BR PE,NA,Amazonas,6448.0


#### Country with largest lake

Construct dataframe of country IDs, country names, lake names, and lake areas

In [375]:
df = pd.DataFrame(columns=('countryID', 'country', 'lake', 'area'))

In [376]:
# add country code, lake name, and lake area to dataframe:
i = 0
root = document.getroot()
for child in root.findall('lake'):
    name = child.find('name').text
    countryID = child.get('country')
    # if area is missing, enter into dataframe as 0
    if child.find('area') == None:
        area = 0
    else:
        area = float(child.find('area').text)
    df.loc[i] = countryID, 'NA', name, area
    i += 1

Identify country with largest lake

In [377]:
df = df.sort_values('area', ascending=False).head(1)

# fill in country name
df.set_index('countryID', inplace = True)

df['country'].update(pd.Series(di))
df

,country,lake,area
countryID,,,
R AZ KAZ IR TM,NA,Caspian Sea,386400.0


#### Country with airport at highest elevation

Construct dataframe of country IDs, country names, airport names, and airport elevations

In [378]:
df = pd.DataFrame(columns=('countryID', 'country', 'airport', 'elevation'))

In [379]:
# add country code, airport name, and airport elevation to dataframe:
i = 0
root = document.getroot()
for child in root.findall('airport'):
    name = child.find('name').text
    countryID = child.get('country')
    if child.find('elevation').text == None:
        elev = 0
    else:
        elev = float(child.find('elevation').text)
    df.loc[i] = countryID, 'NA', name, elev
    i += 1

Identify country with highest airport

In [380]:
df = df.sort_values('elevation', ascending=False).head(1)

# fill in country name
df.set_index('countryID', inplace = True)

df['country'].update(pd.Series(di))
df

,country,airport,elevation
countryID,,,
BOL,Bolivia,El Alto Intl,4063.0
